# Statementextraction
Author: Edward Schmuhl

---

This notebook shows how the implementation of the Statementextraction in combination of using the Conditionidentification

In [1]:
import spacy
# Conditionidentification
path_to_ner = "en_condition_identification_model"
conditionidentification = spacy.load(path_to_ner)

# Spacy model
nlp = spacy.load("en_core_web_trf")

## Statementextraction function

The Conditionidentification will detect conditions and their index position. Every head referencing the condition will be extracted. This operation will be applied recursively on the extracted heads until the ROOT is reached. The extracted words form the statement


In [2]:
def extract_Statement(text):
    returnList = []
    condition_Doc = conditionidentification(text)
    doc = nlp(text)
    
    # Conditionidentification
    conditionList = []
    for i in range(0,len(doc)):
        try:
            if(condition_Doc[i].ent_type_ == "CONDITION" and condition_Doc[i].ent_iob_ == "B"):
                index = i
                
                # Conditionidentification
                condition = condition_Doc[i].text
                conditionList.append(condition)
                for k in range(i,len(doc)):
                    if condition_Doc[k].ent_iob_ == "I":
                        condition += " "+condition_Doc[k].text
                        conditionList.append(condition_Doc[k].text)
                        conditionList.append(condition)
                        if doc[k].pos_ == "NOUN" and doc[i].pos_ != "NOUN":
                            index = k

                # Statementextraction        
                statement = ""
                ancestorList = []
                for ancestor in doc[index].ancestors:
                    if ancestor.text not in conditionList:
                        ancestorList.append(ancestor.text)
                for ancestor in reversed(ancestorList):
                    statement += ancestor+" "
                if statement != "":
                    statement += "condition"
                    returnList.append((statement,condition))
        except IndexError:
            print(i,doc.text)
            continue
    return returnList

## Evaluation

In [3]:
print(extract_Statement("this helped my migraine but worsen headache"))

[('helped condition', 'migraine'), ('helped worsen condition', 'headache')]
